In [20]:
import sys
sys.path.append('../')
import config
import tweepy
import os
import json
import codecs

import pandas as pd
import datetime
from datetime import datetime, timedelta, timezone
import locale

In [2]:
# Clientを返す認証関数
def auth():
    # API KEY
    CK = config.CK
    CS = config.CS
    AT = config.AT
    AS = config.AS
    BT = config.BT

    # tweepy setting
    # auth = tweepy.OAuthHandler(CK, CS)
    # auth.set_access_token(AT, AS)
    

    # wait_on_rate_limit = True とするとレートリミットを確認しながら取得処理を行う
    client = tweepy.Client(bearer_token = BT,\
                           consumer_key = CK,\
                           consumer_secret = CS,\
                           access_token = AT,\
                           access_token_secret = AS,\
                           wait_on_rate_limit = True)

    return client

In [3]:
# Use tweepy.Client()
client = auth()

In [4]:
# Define a search_term for searching tweets
search_term = "CVE"
# 取得したいツイート数
limit = 50000

In [5]:
# https://qiita.com/daisuke-aoki/items/30e3d6e84637326ef1da
# 取得対象のツイートの時間幅を指定する この例では実行前日と前々日の48時間
# iso形式のUTC時間で指定しないと正しく時間指定ができない模様。
# 指定した時間幅に、limitで指定した件数以上のツイートがあってもlimit以上は取得しない
now = datetime.now()
print(now)
start_time = now - timedelta(days=6)
start_time = start_time.replace(hour = 0, minute = 0, second=0, microsecond=0)
print(start_time)
end_time = now.replace(hour = 0, minute = 0, second=0, microsecond=0)
print(end_time)
# now = now.replace(minute=0, second=0, microsecond=0)
end_time_tweepy = str(end_time.isoformat()) +'+09:00'
# start_time = now - timedelta(days=7) 
start_time_tweepy = str(start_time.isoformat())+'+09:00'
print(start_time_tweepy)
print(end_time_tweepy)

2023-02-27 10:21:45.106586
2023-02-21 00:00:00
2023-02-27 00:00:00
2023-02-21T00:00:00+09:00
2023-02-27T00:00:00+09:00


In [6]:
import numpy as np
df = pd.DataFrame(np.arange(12).reshape(3, 4),
                  columns=['col_0', 'col_1', 'col_2', 'col_3'],
                  index=['row_0', 'row_1', 'row_2'])
print(type(df))
print(df)

<class 'pandas.core.frame.DataFrame'>
       col_0  col_1  col_2  col_3
row_0      0      1      2      3
row_1      4      5      6      7
row_2      8      9     10     11


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, row_0 to row_2
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   col_0   3 non-null      int64
 1   col_1   3 non-null      int64
 2   col_2   3 non-null      int64
 3   col_3   3 non-null      int64
dtypes: int64(4)
memory usage: 120.0+ bytes


In [8]:
df.shape

(3, 4)

In [9]:
df_tweet = pd.DataFrame()
for tweet in tweepy.Paginator(client.search_recent_tweets,\
                              query = search_term,\
                              start_time=start_time_tweepy,\
                              end_time=end_time_tweepy,\
                              tweet_fields=['id','created_at','text','author_id','public_metrics',],\
                              max_results = 100).flatten(limit = limit):
    # print(type(tweet.author_id))
    # print(tweet.author_id)
    # print(dir(tweet.author_id))
    # print(type(tweet.data))
    # print(tweet.data)
    # print(dir(tweet.data))
    # print(pd.DataFrame({'id': pd.Series([tweet.id]),\
    #               'created_at':pd.Series([tweet.created_at]),\
    #               'author_id':pd.Series([tweet.author_id]),\
    #               'public_metrics':pd.Series([tweet.public_metrics]),\
    #               'text':pd.Series([tweet.text]),\
    #               'edit_history_tweet_ids':pd.Series([tweet.edit_history_tweet_ids])}\
    #                    ).head)
    df_tweet = pd.concat([df_tweet, pd.DataFrame({'created_at': pd.Series([tweet.created_at]),\
                                                  'id': pd.Series([tweet.id]),\
                                                  'text': pd.Series([tweet.text]),\
                                                  'author_id': pd.Series([tweet.author_id]),\
                                                  'edit_history_tweet_ids': pd.Series([tweet.edit_history_tweet_ids]),\
                                                  'public_metrics': pd.Series([tweet.public_metrics])\
                                                 },\
                                                )])
    #                                             columns = ['id', 'created_at', 'author_id', 'public_metrics', 'text', 'edit_history_tweet_ids'])\
    #                      ])

df_tweet.head()

,created_at,id,text,author_id,edit_history_tweet_ids,public_metrics
0,2023-02-26 14:57:01+00:00,1629858076336463872,@SantiMendezz16 Me han venido los fantasmas de...,1312439595225812993,[1629858076336463872],"{'retweet_count': 0, 'reply_count': 1, 'like_c..."
0,2023-02-26 14:55:35+00:00,1629857717144940544,"$CVE ""ALL THESE LEVELS POSTED AHEAD OF TIME BE...",1543925643284324352,[1629857717144940544],"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
0,2023-02-26 14:53:24+00:00,1629857166155980802,"$CVE ""ALL THESE LEVELS POSTED AHEAD OF TIME BE...",1543925643284324352,[1629857166155980802],"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
0,2023-02-26 14:48:05+00:00,1629855828952899586,RT @elisahategan: Please check out my new arti...,1410559658096574466,[1629855828952899586],"{'retweet_count': 22, 'reply_count': 0, 'like_..."
0,2023-02-26 14:46:04+00:00,1629855320133496832,RT @garquake: Another nice week for CDN Oil pr...,540922293,[1629855320133496832],"{'retweet_count': 7, 'reply_count': 0, 'like_c..."


In [10]:
df_tweet.tail()

,created_at,id,text,author_id,edit_history_tweet_ids,public_metrics
0,2023-02-20 15:03:01+00:00,1627685258719027207,"Admins, patch your Cisco enterprise security s...",14886151,[1627685258719027207],"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
0,2023-02-20 15:02:10+00:00,1627685045782876161,CVE-2022-39952 https://t.co/9uc2QYP28H #Harsia...,1083500716235223040,[1627685045782876161],"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
0,2023-02-20 15:01:10+00:00,1627684792363024391,CVE-2022-1613 https://t.co/3RiHtWk8y1 #HarsiaInfo,1083500716235223040,[1627684792363024391],"{'retweet_count': 0, 'reply_count': 0, 'like_c..."
0,2023-02-20 15:00:51+00:00,1627684713274941440,RT @go_ethereum: Geth version v1.9.24 has been...,769668426,[1627684713274941440],"{'retweet_count': 274, 'reply_count': 0, 'like..."
0,2023-02-20 15:00:06+00:00,1627684525672402947,RT @Raj_Samani: We have posted analysis of CVE...,3090317304,[1627684525672402947],"{'retweet_count': 8, 'reply_count': 0, 'like_c..."


In [11]:
filename = start_time.strftime("%Y%m%d") + "-" + end_time.strftime("%Y%m%d") 
print(filename)

20230221-20230227


In [12]:
df_tweet.to_csv(filename + ".csv", index=False,header=True)

In [13]:
print(len(df_tweet))

13769


In [14]:
print(datetime.now())

2023-02-27 23:22:38.120110


In [15]:
print(datetime.utcnow())

2023-02-27 23:23:05.399105


In [16]:
print(datetime.now().isoformat())

2023-02-27T23:23:53.074576


In [22]:
print(datetime.now(tz=timezone(timedelta(hours=9))))

2023-02-28 08:32:56.744387+09:00


In [24]:
print(datetime.now(tz=timezone(timedelta(hours=9))).isoformat())

2023-02-28T08:34:14.673954+09:00
